<a href="https://colab.research.google.com/github/ArmHPC/Computational-Astrophysics/blob/main/Cloud_Service/test_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clone Repository

In [1]:
!git clone https://github.com/ArmHPC/Computational-Astrophysics.git

Cloning into 'Computational-Astrophysics'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 101 (delta 5), reused 23 (delta 4), pack-reused 72
Receiving objects: 100% (101/101), 94.62 MiB | 15.04 MiB/s, done.
Resolving deltas: 100% (26/26), done.


## Setup Environment

In [2]:
mv ./Computational-Astrophysics/ ./Project/

In [3]:
cd Project

/content/Project


In [4]:
mv ./Cloud_Service/ ./Model/

In [5]:
cd Model/

/content/Project/Model


In [6]:
# !git pull

## Additional Environment Setups

In [7]:
import zipfile
with zipfile.ZipFile('./data.zip', 'r') as zip_ref:
    zip_ref.extractall('')

with zipfile.ZipFile('./Checkpoint.zip', 'r') as zip_ref:
    zip_ref.extractall('')

In [8]:
import sys
sys.path.append('Model/')

## Imports

In [9]:
import os
import shutil

import torch.optim
from torchsummary import summary

import load_data
import models
from Project.Model.test import evaluate
from _helpers import make_directory

from sklearn.metrics import classification_report

## Environment variables

In [10]:
use_gpu = True

In [11]:
# Training Device
if use_gpu:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print('Device: GPU')
else:
    device = torch.device('cpu')
    print('Device: CPU')

Device: GPU


## Data path

In [12]:
# Choose the dataset
num_classes = 5 # Choices: {5, 6, 10}

In [13]:
classes_5 = ['C-H', 'C-N', 'Mrk SB', 'sdA', 'sdB']
classes_6 = ['C-H', 'C-N', 'Mrk Abs', 'Mrk SB', 'sdA', 'sdB']
classes_10 = ['C Ba', 'C-H', 'C-N', 'C-R', 'Mrk Abs', 'Mrk AGN', 'Mrk SB', 'sdA', 'sdB', 'sdO']

assert num_classes in {5, 6, 10}

if num_classes == 10:
    classes = classes_10
    checkpoint_name = 'Dense_10_Focal_25_3_Final/139.pth'
elif num_classes == 5:
    classes = classes_5
    checkpoint_name = 'Dense_5_High_Focal_25_3_Final/59.pth'
else:
    classes = classes_6
    checkpoint_name = 'Dense_6_High_Focal_25_3_Final/136.pth'


In [14]:
# Datasets
data_root = f'./data'

if num_classes != 10:
    new_data_root = f'./data_{num_classes}'
    domains = ['test', 'train']

    for domain in domains:
        make_directory(f'{new_data_root}/{domain}')

        for class_name in classes:
            # make_directory(f'{new_data_root}/{domain}/{class_name}')
            shutil.copytree(f'{data_root}/{domain}/{class_name}/', f'{new_data_root}/{domain}/{class_name}/')

    data_root = new_data_root

In [15]:
test_dir = f'{data_root}/test'
# Optional
train_dir = f'{data_root}/train'

input_shape = (160, 50)

In [16]:
print('Num classes:', num_classes)

Num classes: 5


## Project Parameters

In [17]:
root_dir = os.path.abspath('./')

In [18]:
# Checkpoints are saved in Checkpoint folder
checkpoint_path = f'{root_dir}/Checkpoint/{checkpoint_name}'

## Testing Parameters

In [19]:
# Batch sizes
test_batch_size = 16
# Optional
train_batch_size = 16

## Data Loaders

In [20]:
test_data, test_classes, _ = load_data.load_images(test_dir, test_batch_size, 'test', _drop_last=False)
# Optional
train_data, _, _ = load_data.load_images(train_dir, train_batch_size, 'train', _drop_last=False)

Loading 443 images from ./data_5/test is over.
Loading 2314 images from ./data_5/train is over.


## Training Setup

In [21]:
# Model choices: arch = any(['default', 'default_prev', 'default_bn', 'mobilenet', 'resnet'])
#   default: the proposed network
#   default_bn: similar to the proposed, but with more BatchNorm layers
#   default_prev: the network proposed in the previous work
#   mobilenet: MobileNetV2
#   resnet: Resnet

net = models.Model(num_classes=num_classes, input_shape=input_shape, arch='default').to(device)

### Layers

In [22]:
print(net)

Model(
  (classifier): Classifier(
    (act1): LeakyReLU(negative_slope=0.05)
    (act2): Softmax(dim=1)
    (conv11): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (conv12): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (conv13): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (drop1): Dropout(p=0.2, inplace=False)
    (conv21): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (conv22): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (conv23): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (mp2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (drop2): Dropout(p=0.2, inp

### Output Summary

In [23]:
summary(net, (1, 160, 50))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 32, 160, 50]             320
         LeakyReLU-2          [-1, 32, 160, 50]               0
            Conv2d-3          [-1, 32, 160, 50]           9,248
         LeakyReLU-4          [-1, 32, 160, 50]               0
            Conv2d-5          [-1, 32, 160, 50]           9,248
         LeakyReLU-6          [-1, 32, 160, 50]               0
         MaxPool2d-7           [-1, 32, 80, 25]               0
       BatchNorm2d-8           [-1, 32, 80, 25]              64
           Dropout-9           [-1, 32, 80, 25]               0
           Conv2d-10           [-1, 64, 80, 25]          18,496
        LeakyReLU-11           [-1, 64, 80, 25]               0
           Conv2d-12           [-1, 64, 80, 25]          36,928
        LeakyReLU-13           [-1, 64, 80, 25]               0
           Conv2d-14           [-1, 64,

In [24]:
# Setting the network up for evaluation
net.load_state_dict(torch.load(checkpoint_path))
net.eval()

Model(
  (classifier): Classifier(
    (act1): LeakyReLU(negative_slope=0.05)
    (act2): Softmax(dim=1)
    (conv11): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (conv12): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (conv13): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (drop1): Dropout(p=0.2, inplace=False)
    (conv21): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (conv22): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (conv23): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (mp2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (drop2): Dropout(p=0.2, inp

In [25]:
print('\nEvaluation started:')

train_score = evaluate(dataloader=train_data, model=net, domain='train', device=device)
test_score = evaluate(dataloader=test_data, model=net, device=device, classes=test_classes)


Evaluation started:
Accuracy of the network on the train images: 88.46 %
Accuracy of the network on the test images: 88.71 %
              precision    recall  f1-score   support

         C-H       0.90      0.93      0.92       117
         C-N       0.67      0.57      0.62         7
      Mrk SB       0.91      0.96      0.93       132
         sdA       0.67      0.39      0.49        31
         sdB       0.89      0.90      0.90       156

    accuracy                           0.89       443
   macro avg       0.81      0.75      0.77       443
weighted avg       0.88      0.89      0.88       443

